# 様々なデータをOMMX Artifact形式で共有する

数理最適化のワークフローでは、多様なデータの生成と管理が不可欠です。これらのデータを適切に管理することで、計算結果の再現性が確保され、チーム内での効率的な共有が可能になります。

OMMXは、これらの多様なデータを効率的かつシンプルに管理する仕組みを提供します。具体的には、OMMX Artifactというデータ形式を定義し、最適化計算に関連する多様なデータの保存・管理・共有をOMMX SDKによって可能にします。

## 事前準備：共有するデータ

In [ ]:
from ommx.v1 import Instance, DecisionVariable, Constraint
from ommx_pyscipopt_adapter import instance_to_model, model_to_solution
import pandas as pd

# 0-1ナップサック問題のデータを用意する
data = {
    # 各アイテムの価値
    "v": [10, 13, 18, 31, 7, 15],
    # 各アイテムの重さ
    "w": [11, 15, 20, 35, 10, 33],
    # ナップサックの耐荷重
    "W": 47,
    # アイテムの総数
    "N": 6,
}

# 決定変数を定義する
x = [
    # バイナリ変数 x_i を定義する
    DecisionVariable.binary(
        # 決定変数のIDを指定する
        id=i,
        # 決定変数の名前を指定する
        name="x",
        # 決定変数の添え字を指定する
        subscripts=[i],
    )
    # バイナリ変数を num_items 個だけ用意する
    for i in range(data["N"])
]

# 目的関数を定義する
objective = sum(data["v"][i] * x[i] for i in range(data["N"]))

# 制約条件を定義する
constraint = Constraint(
    # 制約条件の名前
    name = "重量制限",
    # 制約式の左辺を指定する
    function=sum(data["w"][i] * x[i] for i in range(data["N"])) - data["W"],
    # 等式制約 (==0) or 不等式制約 (<=0) を指定する
    equality=Constraint.LESS_THAN_OR_EQUAL_TO_ZERO,
)

# インスタンスを作成する
instance = Instance.from_components(
    # インスタンスに含まれる全ての決定変数を登録する
    decision_variables=x,
    # 目的関数を登録する
    objective=objective,
    # 全ての制約条件を登録する
    constraints=[constraint],
    # 最大化問題であることを指定する
    sense=Instance.MAXIMIZE,
)

# インスタンスをSCIPのモデルに変換する
model = instance_to_model(instance)
# SCIPで解く
model.optimize()
# 計算結果をommx.vi.Solutionとして取得する
solution = model_to_solution(model, instance)

# 最適解の分析をする
df_vars = solution.decision_variables
df = pd.DataFrame.from_dict(
    {
        "アイテムの番号": df_vars.index,
        "ナップサックに入れるか？": df_vars["value"].apply(lambda x: "入れる" if x == 1.0 else "入れない"),
    }
)

In [ ]:
from myst_nb import glue

glue("instance", instance, display=False)
glue("solution", solution, display=False)
glue("data", data, display=False)
glue("df", df, display=False)

```{list-table}
:header-rows: 1
:widths: 5 30 10

* - 変数名
  - 説明
  - 値
* - `instance`
  - 0-1ナップサック問題に対応する `ommx.v1.Instance` オブジェクト
  - ````{toggle}
    ```{glue:} instance
    ```
    ````
* - `solution`
  - 0-1ナップサック問題をSCIPで解いた計算結果が格納されている `ommx.v1.Solution` オブジェクト
  - ````{toggle}
    ```{glue:} solution
    ```
    ````
* - `data`
  - 0-1ナップサック問題の入力データ
  - ```{glue:} data
    ```
* - `df`
  - 0-1ナップサック問題の最適解表す `pandas.DataFrame` オブジェクト
  - {glue:}`df`
```

## ローカルにOMMX Artfactデータを作成する

In [ ]:
from ommx.artifact import ArtifactBuilder

# OMMX Artifactファイルの名前を指定する
filename = "my_instance.ommx"

# 1. OMMX Artifactファイルを作成するためのビルダーを作成する
builder = ArtifactBuilder.new_archive_unnamed(filename)

# 2. ArtifactBuilderにデータを追加する
# ommx.v1.Instance オブジェクトを追加する
builder.add_instance(instance)
# ommx.v1.Solution オブジェクトを追加する
builder.add_solution(solution)
# pandas.DataFrame オブジェクトを追加する
builder.add_dataframe(df, title="ナップサック問題の最適解")
# JSONに変換可能なオブジェクトを追加する
builder.add_json(data, title="ナップサック問題のデータ")

# 3. OMMX Artifactファイルを作成する
builder.build()

In [ ]:
!ls -l "my_instance.ommx"

## ローカルにあるOMMX Artfactデータを読み取る

In [ ]:
from ommx.artifact import Artifact

# ローカルにあるOMMX Artifactファイルを読み込む
artifact = Artifact.load_archive(filename)

In [ ]:
import pandas as pd

# OMMX Artifactファイルに含まれる全てのデータをpandasで表示する
pd.DataFrame(desc.to_dict() for desc in artifact.layers)

In [ ]:
# Remove the created OMMX Artifact file to clean up
!rm my_instance.ommx